In [5]:
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
import psycopg2
from getpass import getpass

In [6]:
password2 = getpass('database passwod')

In [7]:
import pandas.io.sql as psql
connection = psycopg2.connect(
            host="localhost",
            database="final-project",
            user="postgres",
            password=password2)
df = psql.read_sql('SELECT * FROM survey_data', connection)

In [9]:
df.head(5)

,cps19_responseid,cps19_citizenship,cps19_yob,cps19_gender,cps19_province,cps19_education,cps19_demsat,cps19_interest_gen_1,cps19_interest_elxn_1,cps19_v_likely,...,cps19_language_vietnamese,cps19_language_no_answer,cps19_language_aborginal,cps_19_language_other,cps19_employment,cps19_union,cps19_children,cps19_income_number,cps19_marital,cps19_household
0,R_27WeMQ1asip2cMD,Canadian citizen,2000,A woman,Ontario,Completed secondary/ high school,Fairly satisfied,8.0,6.0,Certain to vote,...,No,No,No,No,Working for pay part-time,No,No,56000.0,Never Married,6
1,R_3j7fAVYfVCewi3H,Canadian citizen,2000,A woman,Ontario,Some university,Fairly satisfied,10.0,10.0,Certain to vote,...,No,No,No,No,Student and working for pay,No,No,30000.0,Never Married,5
2,R_brdMqsPTvQ5t1tL,Canadian citizen,2000,A woman,Ontario,Completed secondary/ high school,Fairly satisfied,8.0,8.0,Certain to vote,...,No,No,No,No,Working for pay part-time,No,No,13000.0,Never Married,2
3,R_Wumhl7QEMURFqZH,Canadian citizen,2000,A man,Ontario,Completed secondary/ high school,Fairly satisfied,9.0,8.0,Certain to vote,...,No,No,No,No,Student and working for pay,No,No,55000.0,Never Married,3
4,R_3EH051N9vLmOOHM,Canadian citizen,2000,A woman,Ontario,Completed secondary/ high school,Fairly satisfied,10.0,10.0,Certain to vote,...,No,No,No,No,Student,No,No,190000.0,Never Married,4


In [10]:
df.columns

Index(['cps19_responseid', 'cps19_citizenship', 'cps19_yob', 'cps19_gender',
       'cps19_province', 'cps19_education', 'cps19_demsat',
       'cps19_interest_gen_1', 'cps19_interest_elxn_1', 'cps19_v_likely',
       'cps19_votechoice', 'cps19_fed_gov_sat', 'cps19_lr_scale',
       'cps19_spend_educ', 'cps19_spend_env', 'cps19_spend_just_law',
       'cps19_spend_defence', 'cps19_spend_imm_min', 'cps19_econ_retro',
       'cps19_econ_fed', 'cps19_ownfinanc_fed', 'cps19_imm', 'cps19_refugees',
       'cps19_govt_confusing', 'cps19_govt_say', 'cps19_pol_eth',
       'cps19_lib_promises', 'cps19_snclav', 'cps19_volunteer',
       'cps19_duty_choice', 'cps19_own_fin_retro', 'cps19_prov_gov_sat',
       'cps19_groups_therm_1', 'cps19_groups_therm_2', 'cps19_groups_therm_3',
       'cps19_groups_therm_4', 'cps19_groups_therm_5', 'cps19_turnout_2015',
       'cps19_religion', 'cps19_bornin_canada', 'cps19_ethnicity_first_nat',
       'cps19_ethnicity_british', 'cps19_ethnicity_chinese',
    

In [11]:
columns = list(df.columns)
columns
target = ['cps19_votechoice']

In [12]:
df.describe()

,cps19_yob,cps19_interest_gen_1,cps19_interest_elxn_1,cps19_lr_scale,cps19_groups_therm_1,cps19_groups_therm_2,cps19_groups_therm_3,cps19_groups_therm_4,cps19_groups_therm_5,cps19_income_number,cps19_household
count,13228.000000,13228.00000,13228.000000,13228.000000,13228.000000,13228.000000,13228.000000,13228.000000,13228.000000,1.322800e+04,13228.000000
mean,1969.070910,7.05564,7.613471,5.127986,64.953281,62.503477,62.846462,58.431736,44.706758,9.059620e+04,11.745313
std,16.159103,2.18092,2.133672,2.458060,25.468029,26.214668,27.543003,27.733246,23.515393,1.579430e+05,762.117501
min,1920.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,1.000000
25%,1956.000000,6.00000,7.000000,3.000000,51.000000,50.000000,50.000000,40.000000,29.000000,4.500000e+04,2.000000
50%,1968.000000,7.00000,8.000000,5.000000,69.000000,66.000000,65.000000,61.000000,45.000000,7.500000e+04,2.000000
75%,1983.000000,9.00000,9.000000,7.000000,84.000000,82.000000,85.000000,80.000000,61.000000,1.100000e+05,3.000000
max,2001.000000,10.00000,10.000000,10.000000,100.000000,100.000000,100.000000,100.000000,100.000000,7.696588e+06,72000.000000


In [14]:
X = pd.get_dummies(df, columns=['cps19_responseid', 'cps19_citizenship', 'cps19_yob',
       'cps19_gender', 'cps19_province', 'cps19_education', 'cps19_demsat',
       'cps19_interest_gen_1', 'cps19_interest_elxn_1', 'cps19_v_likely',
       'cps19_votechoice', 'cps19_fed_gov_sat', 'cps19_lr_scale',
       'cps19_spend_educ', 'cps19_spend_env', 'cps19_spend_just_law',
       'cps19_spend_defence', 'cps19_spend_imm_min', 'cps19_econ_retro',
       'cps19_econ_fed', 'cps19_ownfinanc_fed', 'cps19_imm', 'cps19_refugees',
       'cps19_govt_confusing', 'cps19_govt_say', 'cps19_pol_eth',
       'cps19_lib_promises', 'cps19_snclav', 'cps19_volunteer',
       'cps19_duty_choice', 'cps19_own_fin_retro', 'cps19_prov_gov_sat',
       'cps19_groups_therm_1', 'cps19_groups_therm_2', 'cps19_groups_therm_3',
       'cps19_groups_therm_4', 'cps19_groups_therm_5', 'cps19_turnout_2015',
       'cps19_religion', 'cps19_bornin_canada', 'cps19_ethnicity_first_nat',
       'cps19_ethnicity_british', 'cps19_ethnicity_chinese',
       'cps19_ethnicity_dutch', 'cps19_ethnicity_english',
       'cps19_ethnicity_french', 'cps19_ethnicity_french_can',
       'cps19_ethnicity_german', 'cps19_ethnicity_hispanic',
       'cps19_ethnicity_indian', 'cps19_ethnicity_inuk_inuit',
       'cps19_ethnicity_irish', 'cps19_ethnicity_italian',
       'cps19_ethnicity_metis', 'cps19_ethnicity_polish',
       'cps19_ethnicity_quebecois', 'cps19_ethnicity_scottish',
       'cps19_ethnicity_ukranian', 'cps19_ethnicity_no_answer',
       'cps19_ethnicity_other_1', 'cps19_ethnicity_other_2', 'cps19_sexuality',
       'cps19_language_english', 'cps19_language_french',
       'cps19_language_arabic', 'cps19_language_chinese',
       'cps19_language_filipino', 'cps19_language_german',
       'cps19_language_indian', 'cps19_language_italian',
       'cps19_language_korean', 'cps19_language_pakistani',
       'cps19_language_persian', 'cps19_language_russian',
       'cps19_language_spanish', 'cps19_language_tamil',
       'cps19_language_vietnamese', 'cps19_language_no_answer',
       'cps19_language_aborginal', 'cps_19_language_other', 'cps19_employment',
       'cps19_union', 'cps19_children', 'cps19_income_number', 'cps19_marital',
       'cps19_household'])


In [15]:
y = df['cps19_votechoice']

In [16]:
X.describe()

,cps19_responseid_R_006mfFdK5t14ZfX,cps19_responseid_R_00oVLohxA0EQgbn,cps19_responseid_R_00phmxEnhmy3GvL,cps19_responseid_R_00qmWwtHl4CVsBz,cps19_responseid_R_01hJBgFwuFGakmd,cps19_responseid_R_01xyPvm9p6han17,cps19_responseid_R_02PGKxVSjw0jmoh,cps19_responseid_R_02RQeUD92qD3kVH,cps19_responseid_R_02umG7oilzaz6k9,cps19_responseid_R_031EZSaZfpxscg1,...,cps19_household_29,cps19_household_33,cps19_household_34,cps19_household_40,cps19_household_54,cps19_household_69,cps19_household_89,cps19_household_501,cps19_household_50000,cps19_household_72000
count,13228.000000,13228.000000,13228.000000,13228.000000,13228.000000,13228.000000,13228.000000,13228.000000,13228.000000,13228.000000,...,13228.000000,13228.000000,13228.000000,13228.000000,13228.000000,13228.000000,13228.000000,13228.000000,13228.000000,13228.000000
mean,0.000076,0.000076,0.000076,0.000076,0.000076,0.000076,0.000076,0.000076,0.000076,0.000076,...,0.000076,0.000076,0.000076,0.000076,0.000076,0.000076,0.000076,0.000076,0.000076,0.000076
std,0.008695,0.008695,0.008695,0.008695,0.008695,0.008695,0.008695,0.008695,0.008695,0.008695,...,0.008695,0.008695,0.008695,0.008695,0.008695,0.008695,0.008695,0.008695,0.008695,0.008695
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [17]:
y.value_counts()

Liberal Party                       4201
Conservative Party                  3767
ndp                                 1851
Don't know/ Prefer not to answer    1329
Green Party                         1075
Bloc Qu<e9>b<e9>cois                 630
People's Party                       288
Another party (please specify)        87
Name: cps19_votechoice, dtype: int64

In [18]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=28)
Counter(y_train)

Counter({'Liberal Party': 3138,
         'Green Party': 813,
         'Conservative Party': 2861,
         'ndp': 1371,
         "Don't know/ Prefer not to answer": 984,
         'Bloc Qu<e9>b<e9>cois': 481,
         "People's Party": 211,
         'Another party (please specify)': 62})

In [19]:
from sklearn.ensemble import RandomForestClassifier
rf_model = RandomForestClassifier(n_estimators=100, random_state=28)
rf_model = rf_model.fit(X_train, y_train)

In [20]:
from sklearn.metrics import balanced_accuracy_score
y_pred = rf_model.predict(X_test)
acc_score = balanced_accuracy_score(y_test, y_pred)
acc_score

0.6370781185203092

In [21]:
from sklearn.metrics import confusion_matrix 
cm = confusion_matrix(y_test, y_pred)
cm_df = pd.DataFrame(cm, index=["Liberal Party", "Conservative Party", "ndp", "Don't know/ Prefer not to answer", "Green Party", "Bloc Qu<e9>b<e9>cois", "People's Party", "Another Party (please specify)"], columns=["Liberal Party", "Conservative Party", "ndp", "Don't know/ Prefer not to answer", "Green Party", "Bloc Qu<e9>b<e9>cois", "People's Party", "Another party (please specify)"])
cm_df

,Liberal Party,Conservative Party,ndp,Don't know/ Prefer not to answer,Green Party,Bloc Qu<e9>b<e9>cois,People's Party,Another party (please specify)
Liberal Party,1,0,9,0,0,14,0,1
Conservative Party,0,128,5,0,0,16,0,0
ndp,0,0,887,0,0,19,0,0
Don't know/ Prefer not to answer,0,1,39,246,0,59,0,0
Green Party,0,1,47,0,159,54,0,1
Bloc Qu<e9>b<e9>cois,0,0,9,0,0,1054,0,0
People's Party,0,1,57,0,0,13,6,0
Another Party (please specify),0,0,31,0,0,51,0,398
